In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import torch
import open_clip
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm 
from timm import create_model
from torchvision import transforms
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [3]:
file_path = "/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/AllCarabids.csv"
insect_df = pd.read_csv(file_path, sep="\t")
insect_df.rename(columns={"filepath": "ImageFilePath", "class": "ScientificName"}, inplace=True)
insect_df['ImageFilePath'] = "/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/" + insect_df['ImageFilePath']
df = insect_df[['ImageFilePath', 'ScientificName']]
df = df.fillna("Unknown")
df.head(2)

,ImageFilePath,ScientificName
0,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela sexguttata
1,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela sexguttata


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = create_model("convnext_base", pretrained=True).to(device)


preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [5]:
def extract_image_features(image_path) : 
    
    image = Image.open(image_path).convert("RGB")
    
    image_tensor = preprocess(image).unsqueeze(0).to(device)

    with torch.no_grad():
        features = model.forward_features(image_tensor).mean(dim=[2, 3])

    return features.cpu().numpy()
    

In [6]:
# Separate known and unknown samples
df_known = df[df["ScientificName"] != "Unknown"]
df_unknown = df[df["ScientificName"] == "Unknown"]

# Extract features for known samples (only passing img)
X_known = np.vstack([extract_image_features(img) for img in tqdm(df_known["ImageFilePath"], desc="Processing ...")])

# Encode labels only for known species
le = LabelEncoder()
y_known = le.fit_transform(df_known["ScientificName"])

df_indices_known = df_known.index

# Train-test split only on known species
X_train, X_test_known, y_train, y_test_known, train_idx, test_idx_known = train_test_split(
    X_known, y_known, df_indices_known, test_size=0.2, random_state=42
)

# Extract features for unknown samples (only passing img)
X_test_unknown = np.vstack([extract_image_features(img) for img in tqdm(df_unknown["ImageFilePath"], desc="Processing Unknown ...")])
test_idx_unknown = df_unknown.index

# Assign a placeholder label (-1) for unknown samples
y_test_unknown = np.full(len(X_test_unknown), -1)

# Combine known and unknown test sets
X_test = np.vstack([X_test_known, X_test_unknown])
y_test = np.concatenate([y_test_known, y_test_unknown])
test_idx = np.concatenate([test_idx_known, test_idx_unknown])

print(f"Train set: {X_train.shape}, Test set: {X_test.shape}")


Processing ...:   0%|          | 0/96530 [00:00<?, ?it/s]

Processing Unknown ...:   0%|          | 0/4355 [00:00<?, ?it/s]

Train set: (77224, 1024), Test set: (23661, 1024)


In [7]:
test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)

seed = 99


models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=11),  
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}


for name, model in tqdm(models.items(), desc="Training Models", leave=True):
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds

    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)

    metrics[name] = {"Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}

    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


Training Models:   0%|          | 0/4 [00:00<?, ?it/s]

NaiveBayes                | Acc: 37.80% | Prec: 41.81% | Rec: 37.80% | F1: 37.11% | Bal Acc: 27.34% | MCC: 0.3835
LogisticRegression        | Acc: 65.76% | Prec: 60.60% | Rec: 65.76% | F1: 62.14% | Bal Acc: 47.90% | MCC: 0.6682
NearestNeighbor           | Acc: 51.03% | Prec: 48.19% | Rec: 51.03% | F1: 48.01% | Bal Acc: 28.82% | MCC: 0.5180
MLP_Baseline              | Acc: 64.27% | Prec: 59.72% | Rec: 64.27% | F1: 61.18% | Bal Acc: 44.33% | MCC: 0.6530


In [8]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_NearestNeighbor,Pred_MLP_Baseline
19596,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela repanda,Parvindela terricola,Cicindela oregona,Cicindela repanda,Cicindela oregona
18754,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Agonum aeruginosum,Lebia viridis,Agonum aeruginosum,Calosoma semilaeve,Bembidion obtusum


In [9]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/AllCarabids/4.ConvNeXt-AllCarabids-species.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/AllCarabids/4.ConvNeXt-AllCarabids-species-metrics.csv", index=False)